In [1]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm

In [2]:
files = []
for file in os.listdir('odi'):
    files.append(os.path.join('odi',file))

In [3]:
main_df = pd.DataFrame()
cnt = 1
for file in tqdm(files):
    with open(file,'r') as f:
        df = pd.json_normalize(safe_load(f))
        df['match_id'] = cnt
        main_df = pd.concat([df,main_df],ignore_index=True)
        cnt+=1

100%|█████████▉| 2822/2823 [23:16<00:00,  2.02it/s]


NotImplementedError: 

In [6]:
main_df.columns

Index(['innings', 'meta.data_version', 'meta.created', 'meta.revision',
       'info.balls_per_over', 'info.city', 'info.dates', 'info.gender',
       'info.match_type', 'info.outcome.by.wickets',
       ...
       'info.registry.people.TS Roland-Jones',
       'info.registry.people.IA Thomson', 'info.registry.people.Z Nizami',
       'info.registry.people.K Christie', 'info.registry.people.SD Heazlett',
       'info.registry.people.Tanbir Hayder',
       'info.registry.people.Subashis Roy', 'info.registry.people.FY Fazal',
       'info.registry.people.KK Nair', 'info.registry.people.RAJ Smith'],
      dtype='object', length=2935)

In [7]:
main_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.toss.decision',
    'info.umpires',
    'info.supersubs.South Africa',
    'info.supersubs.West Indies',
    'info.supersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets',
    'info.supersubs.Bangladesh', 
    'info.supersubs.Pakistan',
    'info.supersubs.Sri Lanka', 
    'info.supersubs.India',
    'info.supersubs.Australia', 
    'info.supersubs.South Africa',
    'info.supersubs.England', 
    'info.supersubs.ICC World XI',
    'info.supersubs.Zimbabwe', 
    'info.supersubs.Africa XI',
    'info.supersubs.Asia XI', 
    'info.outcome.eliminator',
],inplace=True)

In [8]:
main_df['info.gender'].value_counts()

info.gender
male      2422
female     400
Name: count, dtype: int64

In [9]:
main_df = main_df[main_df['info.gender'] == 'male']
main_df.drop(columns=['info.gender'],inplace=True)

In [10]:
main_df['info.match_type'].value_counts()

info.match_type
ODI    2422
Name: count, dtype: int64

In [11]:
main_df['info.overs'].value_counts()

info.overs
50    2422
Name: count, dtype: int64

In [12]:
main_df.drop(columns=['info.match_type','info.overs'],inplace=True)

In [15]:
main_df = main_df[['innings', 
       'info.city', 'info.dates',
        'info.outcome.winner', 
       'info.teams', 
       'info.venue', 'match_id',
    ]]

In [16]:
count = 0
delivery_df = pd.DataFrame()
for index, row in main_df.iterrows():
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    temp_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    main_df = pd.concat([temp_df,main_df],ignore_index=True)

In [17]:
main_df

,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,innings,info.city,info.dates,info.outcome.winner,info.teams,info.venue
0,2422,"[Australia, Pakistan]",Australia,0.1,DA Warner,Mohammad Amir,0.0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba",NaN,NaN,NaN,NaN,NaN,NaN
1,2422,"[Australia, Pakistan]",Australia,0.2,DA Warner,Mohammad Amir,0.0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba",NaN,NaN,NaN,NaN,NaN,NaN
2,2422,"[Australia, Pakistan]",Australia,0.3,DA Warner,Mohammad Amir,0.0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba",NaN,NaN,NaN,NaN,NaN,NaN
3,2422,"[Australia, Pakistan]",Australia,0.4,DA Warner,Mohammad Amir,0.0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba",NaN,NaN,NaN,NaN,NaN,NaN
4,2422,"[Australia, Pakistan]",Australia,0.5,DA Warner,Mohammad Amir,1.0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702593,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'1st innings': {'team': 'Australia', 'delive...",NaN,[2017-01-26],Australia,"[Australia, Pakistan]",Adelaide Oval
702594,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'1st innings': {'team': 'Australia', 'delive...",NaN,[2017-01-22],Australia,"[Australia, Pakistan]",Sydney Cricket Ground
702595,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'1st innings': {'team': 'Pakistan', 'deliver...",Perth,[2017-01-19],Australia,"[Australia, Pakistan]",Western Australia Cricket Association Ground
702596,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'1st innings': {'team': 'Australia', 'delive...",NaN,[2017-01-15],Pakistan,"[Australia, Pakistan]",Melbourne Cricket Ground


In [19]:
main_df.drop(columns=['innings','info.city','info.dates','info.outcome.winner','info.teams','info.venue'],inplace=True)

In [20]:
main_df['city'] = main_df['city'].fillna(0)

In [21]:
main_df = main_df.dropna()

In [22]:
main_df.isnull().sum()

match_id            0
teams               0
batting_team        0
ball                0
batsman             0
bowler              0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [23]:
bowling_team = []
for index,row in main_df.iterrows():
    for team in row['teams']:
        if team != row['batting_team']:
            bowling_team.append(team)

In [24]:
main_df['bowling_team'] = bowling_team

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\3421552975.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['bowling_team'] = bowling_team


In [25]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka', 
    'Netherlands'  
]

In [26]:
main_df = main_df[main_df['batting_team'].isin(teams)]
main_df = main_df[main_df['bowling_team'].isin(teams)]

In [27]:
main_df.shape

(487744, 11)

In [28]:
final = main_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [29]:
cities = np.where(final['city'] == 0, final['venue'].str.split().apply(lambda x:x[0]), final['city'])

In [30]:
final['city'] = cities

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\3422534025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['city'] = cities


In [31]:
final.drop(columns=['venue'],inplace=True)

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\2320069556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.drop(columns=['venue'],inplace=True)


In [32]:
total_df = final.groupby('match_id').sum()['runs'].reset_index()
final = final.merge(total_df,on='match_id')

In [33]:
con = {'runs_x': int,'runs_y': int}
final = final.astype(con)

In [40]:
curscore = []
mid = 2422
sum = 0
for ind,row in final.iterrows():
    if(row['match_id'] == mid):
      sum += row['runs_x']
      curscore.append(sum)
      continue
    else:
      mid -= 1
      sum = row['runs_x']
      curscore.append(sum)

In [41]:
final['current_score'] = curscore

In [36]:
final['over'] = final['ball'].apply(lambda x:str(x).split(".")[0])
final['ball_no'] = final['ball'].apply(lambda x:str(x).split(".")[1])

In [37]:
final['balls_bowled'] = (final['over'].astype('int')*6) + final['ball_no'].astype('int')

In [38]:
final['crr'] = round((final['current_score']*6)/final['balls_bowled'],2)

In [39]:
final['player_dismissed'] = final['player_dismissed'].apply(lambda x : 0 if x=='0' else 1)

In [42]:
wicketsleft = []
mid = 2422
sum = 0
for ind,row in final.iterrows():
    if(row['match_id'] == mid):
      sum += row['player_dismissed']
      wicketsleft.append(sum)
      continue
    else:
      mid -= 1
      sum = row['player_dismissed']
      wicketsleft.append(sum)

In [43]:
final['wicketsleft'] = 10 - final['player_dismissed']

In [44]:
final_df = final[['match_id','batting_team','bowling_team','runs_x','current_score','balls_bowled','wicketsleft','crr','city','runs_y']]


In [45]:
final_df['balls_left'] = 300 - final_df['balls_bowled']
final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\3541939152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['balls_left'] = 300 - final_df['balls_bowled']
C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\3541939152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)


In [46]:
final_df['crr'] = round((final_df['current_score']*6)/final_df['balls_bowled'],2)

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\1891013313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['crr'] = round((final_df['current_score']*6)/final_df['balls_bowled'],2)


In [47]:
final_df.drop(columns=['balls_bowled'],inplace=True)

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\2226683702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns=['balls_bowled'],inplace=True)


In [48]:
groups = final_df.groupby('match_id')
match_ids = final_df['match_id'].unique()
last_five = []
for id in match_ids:
    sum = groups.get_group(id)['runs_x'].rolling(window=30, min_periods=1).sum()
    last_five.extend(sum)

In [49]:
final_df['last_n'] = last_five

C:\Users\dhair\AppData\Local\Temp\ipykernel_32880\920023651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['last_n'] = last_five


In [50]:
final_df = final_df.astype({'last_n': int})

In [ ]:
final_df.to_csv('final.csv')